# CSCE 633 :: Machine Learning :: Texas A&M University :: Spring 2022

# Programming Assignment 5 (PA 5) + Competition
- **100 points**
- **Due Wednesday, May 04, 11:59 pm**


**Name:**  
**UIN:**

### Instructions
- **NO LATE DAYS ALLOWED FOR THIS ASSIGNMENT**
- You're free to edit this file as you like, although we highly recommend just filling the sections
- Once you've filled out your solutions, submit the notebook on Canvas.
- Do **NOT** forget to type in your name and UIN at the beginning of the notebook.
- For further instructions (with using kaggle) please refer to the course webpage.

# Convolutional Neural Networks

In this assignment, you'll be coding up a convolutional neural network from scratch to classify images using PyTorch.  

### Instructions
- The maximum number of parameters you are allowed to use for your network is **100,000**. 
- You are required to complete the functions defined in the code blocks following each question. Fill out sections of the code marked `"YOUR CODE HERE"`.
- You're free to add any number of methods within each class.
- You may also add any number of additional code blocks that you deem necessary. 
- Once you've filled out your solutions, submit the notebook on Canvas following the instructions [here](https://people.engr.tamu.edu/guni/csce421/assignments.html).
- Do **NOT** forget to type in your name and UIN at the beginning of the notebook.
- Make sure the notebook runs on google colab **WITHOUT** any issues when all cells are ran sequentially (includes installation of libraries). Points might be deducted if there are any bugs present.

## Install Dependencies

In [ ]:
!pip install torchinfo

In [ ]:
# 
# Checking if hardware acceleration enabled
import os 
if int(os.environ['COLAB_GPU']) > 0:
  print ("*** GPU connected")
else:
  print ("*** No hardware acceleration: change to GPU under Runtime > Change runtime type > Hardware accelerator")

## Data Preparation

In [ ]:
# Importing the libraries
import os
import torch
import torchvision
from torchvision.utils import make_grid
from PIL import Image
import requests

import numpy as np

In this assignment, we will use the Fashion-MNIST dataset. Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.  

### Data

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255.  

### Labels

Each training and test example is assigned to one of the following labels:

| Label | Description |
|-------|-------------|
| 0     | T-shirt/top |
| 1     | Trouser     |
| 2     | Pullover    |
| 3     | Dress       |
| 4     | Coat        |
| 5     | Sandal      |
| 6     | Shirt       |
| 7     | Sneaker     |
| 8     | Bag         |
| 9     | Ankle boot  |

Fashion-MNIST is included in the `torchvision` library.

In [ ]:
from torchvision.datasets import FashionMNIST
from torchvision.transforms import Compose, ToTensor, Normalize

In [ ]:
# Transform to normalize the data and convert to a tensor
transform = Compose([ToTensor(),
    Normalize((0.5,), (0.5,))
    ])

# Download the data
dataset = FashionMNIST('MNIST_data/', download = True, train = True, transform = transform)

**NOTE:** You may add more operations to `Compose` if you're performing data augmentation.

## Data Exploration

Let's take a look at the classes in our dataset.

In [ ]:
print(dataset.classes)

In [ ]:
import matplotlib.pyplot as plt

def show_example(img, label):
    print('Label: {} ({})'.format(dataset.classes[label], label))
    plt.imshow(img.squeeze(), cmap='Greys_r')
    plt.axis(False)

In [ ]:
show_example(*dataset[20])

In [ ]:
show_example(*dataset[20000])

## Question 1

## Creating Training and Validation Datasets

The `split_indices` function takes in the size of the entire dataset, `n`, the fraction of data to be used as validation set, `val_frac`, and the random seed and returns the indices of the data points to be added to the validation dataset.  

**Choose a suitable fraction for your validation set and experiment with the seed. Remember that the better your validation set, the higher the chances that your model would do well on the test set.**

In [ ]:
def split_indices(n, val_frac, seed):
    # Determine the size of the validation set
    n_val = int(val_frac * n)
    np.random.seed(seed)
    # Create random permutation between 0 to n-1
    idxs = np.random.permutation(n)
    # Pick first n_val indices for validation set
    return idxs[n_val:], idxs[:n_val]

In [ ]:
######################
#   YOUR CODE HERE   #
######################
val_frac =  0.25 ## Set the fraction for the validation set
rand_seed =  221 ## Set the random seed

train_indices, val_indices = split_indices(len(dataset), val_frac, rand_seed)
print("#samples in training set: {}".format(len(train_indices)))
print("#samples in validation set: {}".format(len(val_indices)))

Next, we make use of the built-in dataloaders in PyTorch to create iterables of our our training and validation sets. This helps in avoiding fitting the whole dataset into memory and only loads a batch of the data that we can decide. 

**Set the `batch_size` depending on the hardware resource (GPU/CPU RAM) you are using for the assignment.**

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

In [ ]:
######################
#   YOUR CODE HERE   #
######################
batch_size = 18 ## Set the batch size

In [ ]:
# Training sampler and data loader
train_sampler = SubsetRandomSampler(train_indices)
train_dl = DataLoader(dataset,
                     batch_size,
                     sampler=train_sampler)

# Validation sampler and data loader
val_sampler = SubsetRandomSampler(val_indices)
val_dl = DataLoader(dataset,
                   batch_size,
                   sampler=val_sampler)

Plot images in a sample batch of data.

In [ ]:
def show_batch(dl):
    for images, labels in dl:
        print (images.size())
        fig, ax = plt.subplots(figsize=(10,10))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, 8).permute(1, 2, 0), cmap='Greys_r')
        break

In [ ]:
show_batch(train_dl)

## Question 2

## Building the Model

**Create your model by defining the network architecture in the `ImageClassifierNet` class.**  
**NOTE:** The number of parameters in your network must be $\leq$ 100,000.

In [ ]:
# Import the libraries
import torch.nn as nn
import torch.nn.functional as F

from torchinfo import summary

In [ ]:
from torchsummary import summary

In [ ]:
from torch.nn.modules.pooling import MaxPool1d
class ImageClassifierNet(nn.Module):
    def __init__(self, n_channels=1, num_classes=10):
        super(ImageClassifierNet, self).__init__()
        ######################
        #   YOUR CODE HERE   #
        ######################

        self.C1 = nn.Conv2d(1,16,kernel_size=2,padding=1)
        self.C2 = nn.Conv2d(16,64,kernel_size=2)
        self.C3 = nn.Conv2d(64,32,kernel_size=3)
        self.C4 = nn.Conv2d(32,16,kernel_size=2,padding=1)
        self.D1 = nn.Dropout(0.3)
        self.D2 = nn.Dropout(0.6)
        self.F1 = nn.Linear(576,72)
        self.F2 = nn.Linear(72,num_classes)
        self.MP1 = nn.MaxPool2d(kernel_size=3,stride=2)
        self.MP2= nn.MaxPool2d(kernel_size=3,stride=2)
        self.MP3= nn.MaxPool2d(kernel_size=2,stride=1)

        self.POOL = nn.AdaptiveAvgPool2d((6, 6))
        
        
    def forward(self, x):
        ######################
        #   YOUR CODE HERE   #
        ######################
        x = self.C1(x)
        x=F.relu(x)
        x=self.MP1(x)
        x=self.C2(x)
        x=F.relu(x)
        x=self.MP2(x)
        x=self.C3(x)
        x=F.relu(x)
        x=self.MP3(x)
        x=self.C4(x)
        x=F.relu(x)
        x=self.POOL(x)
        x=torch.flatten(x, 1)
        x=self.D1(x)
        x=self.F1(x)
        x=self.D2(x)
        x=F.relu(x)
        x=self.F2(x)
        return x

In [ ]:
model = ImageClassifierNet()

The following code block prints your network architecture. It also shows the total number of parameters in your network (see `Total params`).  

**NOTE: The total number of parameters in your model should be <= 100,000.**

In [ ]:
summary(model, input_size=(1, 28, 28))

In [ ]:
(batch_size, 1, 28, 28)

## Enable training on a GPU

In [ ]:
def get_default_device():
    """Use GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)
    
    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()

train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

to_device(model, device)

In [ ]:
device

## Question 3 

## Train the model

**Complete the `train_model` function to train your model on a dataset. Tune your network architecture and hyperparameters on the validation set.**

In [ ]:
def train_model(n_epochs, model, train_dl, val_dl, loss_fn, opt_fn, lr):
    """
    Trains the model on a dataset.
    
    Args:
        n_epochs: number of epochs
        model: ImageClassifierNet object
        train_dl: training dataloader
        val_dl: validation dataloader
        loss_fn: the loss function
        opt_fn: the optimizer
        lr: learning rate
    
    Returns:
        The trained model. 
        A tuple of (model, train_losses, val_losses, train_accuracies, val_accuracies)
    """
    # Record these values the end of each epoch
    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []
    
    ######################
    #   YOUR CODE HERE   #
    ######################

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0.
        for batch_idx, (data, target) in enumerate(train_dl):
            data, target = data.to(device), target.to(device)
            opt_fn.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            opt_fn.step()
            item_loss = loss.item()
            train_loss = train_loss + item_loss

            t_loss = train_loss/(batch_size*len(train_dl))

        train_losses.append(t_loss)

        model.eval()

        train_correct = 0
        for batch_idx, (data, target) in enumerate(train_dl):
            data, target = data.to(device), target.to(device)
            output = model(data)
            output = output.argmax(dim=1, keepdim=True).squeeze()
            T_cor = (output==target).sum().item()
            train_correct += T_cor

            t_acc = (train_correct/(batch_size*len(train_dl)))

        train_accuracies.append(t_acc)

        if len(val_dl) > 0:
            val_loss = 0
            val_correct = 0
            for batch_idx, (data, target) in enumerate(val_dl):
                data, target = data.to(device), target.to(device)
                output = model(data)
                v_loss = loss_fn(output, target).item()
                val_loss += v_loss
                output = output.argmax(dim=1, keepdim=True).squeeze()

                v_corr = (output==target).sum().item()
          
                val_correct += v_corr
            
            val_losses.append(val_loss/(batch_size*len(val_dl)))
            val_accuracies.append(val_correct/(batch_size*len(val_dl)))

            print("Epoch : {}, Train Loss : {:.4f}, Train Accuracy : {:.4f}, Val Loss : {:.4f}, Val Accuracy : {:.4f}".format(epoch, train_loss/(batch_size*len(train_dl)), train_correct/(batch_size*len(train_dl)), 
                  val_loss/(batch_size*len(val_dl)), val_correct/(batch_size*len(val_dl))))
        else:
          print("Epoch : {}, Train Loss : {:.4f}, Train Accuracy : {:.4f}".format(epoch, train_loss/(batch_size*len(train_dl)), train_correct/(batch_size*len(train_dl))))

    
    return model, train_losses, val_losses, train_accuracies, val_accuracies

In [ ]:
len(val_dl)

In [ ]:
# input = torch.randn(16, 1, 28, 28)
# output = torch.randn(16,)
# pred = model(input).argmax(dim=1, keepdim=True).squeeze()
# (pred==output).sum().item()

**Set the maximum number of training epochs, the loss function, the optimizer, and the learning rate.**

In [ ]:
######################
#   YOUR CODE HERE   #
######################
num_epochs = 40  # Max number of training epochs
loss_fn = nn.CrossEntropyLoss()  # Define the loss function
opt_fn =  torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # Select an optimizer
lr =  0.001  # Set the learning rate

In [ ]:
history = train_model(num_epochs, model, train_dl, val_dl, loss_fn, opt_fn, lr)

# (Optional)
# Once training is finished, save model as .pth and avoid retraining for the following blocks

In [ ]:
len(train_dl), len(val_dl)

In [ ]:
# (Optional)
# Add necessary codes to the next block to load the model from file.
# load model history

In [ ]:
model, train_losses, val_losses, train_accuracies, val_accuracies = history

## Plot loss and accuracy

In [ ]:
def plot_accuracy(train_accuracies, val_accuracies):
    """Plot accuracies"""
    plt.plot(train_accuracies, "-x")
    plt.plot(val_accuracies, "-o")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(["Training", "Validation"])
    plt.title("Accuracy vs. No. of epochs")

In [ ]:
plot_accuracy(train_accuracies, val_accuracies)

In [ ]:
def plot_losses(train_losses, val_losses):
    """Plot losses"""
    plt.plot(train_losses, "-x")
    plt.plot(val_losses, "-o")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(["Training", "Validation"])
    plt.title("Loss vs. No. of Epochs")

In [ ]:
plot_losses(train_losses, val_losses)

## Train a model on the entire dataset

In [ ]:
indices, _ = split_indices(len(dataset), 0, rand_seed)

sampler = SubsetRandomSampler(indices)
dl = DataLoader(dataset, batch_size, sampler=sampler)
dl = DeviceDataLoader(dl, device)

**Set the maximum number of training epochs and the learning rate for finetuning your model.**

In [ ]:
######################
#   YOUR CODE HERE   #
######################
num_epochs = 15 # Max number of training epochs
lr = 0.05 # Set the learning rate

In [ ]:
history = train_model(num_epochs, model, dl, [], loss_fn, opt_fn, lr)
model = history[0]

## Check Predictions

In [ ]:
def view_prediction(img, label, probs, classes):
    """
    Visualize predictions.
    """
    probs = probs.cpu().numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(8,15), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).cpu().numpy().squeeze(), cmap='Greys_r')
    ax1.axis('off')
    ax1.set_title('Actual: {}'.format(classes[label]))
    ax2.barh(np.arange(10), probs)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(classes, size='small');
    ax2.set_title('Predicted: probabilities')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

In [ ]:
# Calculate the class probabilites (log softmax) for img
images = iter(dl)
for imgs, labels in images:
    with torch.no_grad():
        model.eval()
        # Calculate the class probabilites (log softmax) for img
        probs = torch.nn.functional.softmax(model(imgs[0].unsqueeze(0)), dim=1)
        # Plot the image and probabilites
        view_prediction(imgs[0], labels[0], probs, dataset.classes)
    break

## Save the model

In [ ]:
# Very important
torch.save(model, 'model')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Question 4

## Compute accuracy on the test set

In [ ]:
test_dataset = FashionMNIST('MNIST_data/', download = True, train = False, transform = transform)

In [ ]:
test_dl = DataLoader(test_dataset, batch_size)
test_dl = DeviceDataLoader(test_dl, device)

In [ ]:
def evaluate(model, test_dl):
    """
    Evaluates your model on the test data.
    
    Args:
        model: ImageClassifierNet object
        test_dl: test dataloader
    
    Returns: 
        Test accuracy.
    """
    ######################
    #   YOUR CODE HERE   #
    ######################
    model.eval()
    test_correct = 0
    for batch_idx, (data, target) in enumerate(test_dl):
        data, target = data.to(device), target.to(device)
        output = model(data)
        output = output.argmax(dim=1, keepdim=True).squeeze()
        test_correct += (output==target).sum().item()
    
    return test_correct/(batch_size*len(test_dl))

In [ ]:
print("Test Accuracy = {:.4f}".format(evaluate(model, test_dl)))

## Preparing the CSV for Kaggle submission

In [ ]:
import zipfile
from tqdm import tqdm
from torchvision.io import read_image
!wget --no-check-certificate \
    "https://people.tamu.edu/~sumedhpendurkar/csce633/test_private/dataset.zip" \
    -O "/tmp/dataset.zip"
zip_ref = zipfile.ZipFile('/tmp/dataset.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
from torch.utils.data import Dataset
import pandas as pd
from tqdm import tqdm
class PrivateImageDataset(Dataset):
    def __init__(self, base_path, length, transform=None):
        self.base_path = base_path
        self.transform = transform
        self.length = length

    def __len__(self):
        return (self.length)

    def __getitem__(self, idx):
        path = self.base_path + str(idx) + '.png'
        im = Image.open(path)
        if self.transform:
            image = self.transform(im)
        return image, 0

In [ ]:
def get_test_labels(model, dataset_size):
    test_dataset = PrivateImageDataset('/tmp/', dataset_size, transform=transform)
    test_dl = DataLoader(test_dataset, batch_size)
    test_dl = DeviceDataLoader(test_dl, device)
    preds, labels = [], []
    with torch.no_grad():
        model.eval()
        
        for i, data in tqdm(enumerate(test_dl)):
            xb, yb = data
            y_pred = model(xb)
            _, y = torch.max(y_pred, dim=1)
            preds.extend(y_pred)
            labels.extend(y)
    return preds, labels

In [ ]:
def create_csv_for_kaggle(model):
    dataset_size = 10000
    _, labels = get_test_labels(model, dataset_size)
    data = []
    for i in range(len(labels)):
        data.append([str(i) + '.png', labels[i].item()])
    df = pd.DataFrame(data, columns=['id', 'label'])
    df.to_csv('submission.csv',index=False)
    return df
    


In [ ]:
create_csv_for_kaggle(model)

## Tips to increase the test accuracy

- **Data augmentation:** Diversifies your training set and leads to better generalization
    - Flipping
    - Rotation
    - Shifting
    - Cropping
    - Adding noise
    - Blurring
    
- **Regularization:** Reduces overfitting on the training set
    - Early stopping
    - Dropout
    - $l_2$ regularization
    - Batch normalization

- **Hyperparameter tuning:**
    - Weight initialization
    - Learning rate
    - Activation functions
    - Optimizers